In [5]:
import os
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm
from collections import Counter

# ==============================
# CONFIG
# ==============================
DATA_DIR = r"E:\ASL_Citizen\NEW\Top_Classes_Landmarks_Preprocessed_method2"
TARGET_FRAMES = 157
# Hands + Face only: FACE_SIZE=180, LH=63, RH=63 => 180+63+63=306
FEATURE_DIM = 306  
BATCH_SIZE = 16
LR = 1e-3
EPOCHS = 60
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
PATIENCE = 7  # Early stopping
print("Using device:", DEVICE)

# ==============================
# DATASET
# ==============================
class LandmarkDataset(Dataset):
    def __init__(self, files, labels, transform=None):
        self.files = files
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        x_path = self.files[idx]
        x = np.load(x_path).astype(np.float32)  # (frames, features)

        # --- FILTER HANDS + FACE ONLY ---
        # Original layout: [pose | face | lh | rh]
        # FACE: 0:180, LH:180:243, RH:243:306 if original FEATURE_DIM=438
        face = x[:, 132:312]  # face portion (adjusted for your original 438)
        lh = x[:, 312:375]    # left hand
        rh = x[:, 375:438]    # right hand
        x = np.concatenate([face, lh, rh], axis=1)  # (frames, 306)

        # Optional augmentation: small jitter
        if self.transform:
            x = self.transform(x)

        x = torch.tensor(x).permute(1, 0)  # (features, frames)
        y = torch.tensor(self.labels[idx], dtype=torch.long)
        return x, y

# ==============================
# LOAD FILES & LABELS
# ==============================
files, labels = [], []

for f in os.listdir(DATA_DIR):
    if f.endswith(".npy") and "_mask" not in f:
        files.append(os.path.join(DATA_DIR, f))
        name = f.rsplit("_", 1)[0]
        labels.append(name.split("_")[0])

print(f"Found {len(files)} samples")

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(labels)

# ==============================
# FILTER CLASSES WITH <2 SAMPLES
# ==============================
label_counts = Counter(y_encoded)
filtered_indices = [i for i, y in enumerate(y_encoded) if label_counts[y] >= 2]

files = [files[i] for i in filtered_indices]
y_encoded = y_encoded[filtered_indices]

# Re-encode after filtering
le = LabelEncoder()
y_encoded = le.fit_transform(y_encoded)
num_classes = len(le.classes_)
print(f"Number of gloss classes after filtering: {num_classes}")

# ==============================
# TRAIN/VAL/TEST SPLIT
# ==============================
files_train, files_tmp, y_train, y_tmp = train_test_split(
    files, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)
files_val, files_test, y_val, y_test = train_test_split(
    files_tmp, y_tmp, test_size=0.5, stratify=y_tmp, random_state=42
)

# ==============================
# CLASS WEIGHTS & LOSS
# ==============================
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(num_classes),
    y=y_train
)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32).to(DEVICE)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)

# ==============================
# DATASET & DATALOADERS
# ==============================
train_dataset = LandmarkDataset(files_train, y_train)
val_dataset = LandmarkDataset(files_val, y_val)
test_dataset = LandmarkDataset(files_test, y_test)

# WeightedRandomSampler for class balancing
sample_weights = np.array([class_weights[y] for y in y_train])
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, sampler=sampler)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ==============================
# CNN MODEL (unchanged)
# ==============================
class CNN1D(nn.Module):
    def __init__(self, input_features, num_classes):
        super(CNN1D, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv1d(input_features, 256, kernel_size=3, padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.1),

            nn.Conv1d(256, 128, kernel_size=3, padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.05),

            nn.Conv1d(128, 64, kernel_size=3, padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1)
        )
        self.fc = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.conv_block(x)
        x = x.squeeze(-1)
        x = self.fc(x)
        return x

model = CNN1D(FEATURE_DIM, num_classes).to(DEVICE)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='max', factor=0.5, patience=3
)
print(model)


# ==============================
# TRAINING LOOP WITH EARLY STOPPING
# ==============================
def train_model(model, train_loader, val_loader, epochs, patience):
    best_val_acc = 0.0
    counter = 0

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0

        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]"):
            x, y = x.to(DEVICE), y.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * x.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == y).sum().item()
            total += y.size(0)

        train_loss = running_loss / total
        train_acc = correct / total

        # Validation
        model.eval()
        val_correct = 0
        val_total = 0
        with torch.no_grad():
            for x_val, y_val in val_loader:
                x_val, y_val = x_val.to(DEVICE), y_val.to(DEVICE)
                outputs = model(x_val)
                preds = outputs.argmax(dim=1)
                val_correct += (preds == y_val).sum().item()
                val_total += y_val.size(0)
        val_acc = val_correct / val_total

        # Step scheduler and print if LR changed
        old_lr = optimizer.param_groups[0]['lr']
        scheduler.step(val_acc)
        new_lr = optimizer.param_groups[0]['lr']
        if new_lr != old_lr:
            print(f"  🔹 LR reduced from {old_lr:.6f} to {new_lr:.6f}")

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            counter = 0
            torch.save(model.state_dict(), "CNN_no_SW_hands_and_face_only_best_model.pth")
            print("  🔹 Saved best model")
        else:
            counter += 1
            if counter >= patience:
                print("Early stopping triggered!")
                break

    print(f"Training finished. Best val acc: {best_val_acc:.4f}")

# ==============================
# EVALUATION
# ==============================
def evaluate_model(model, test_loader, label_encoder):
    # Load best model
    model.load_state_dict(torch.load("CNN_no_SW_hands_and_face_only_best_model.pth"))
    model.eval()
    
    all_preds, all_labels = [], []

    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(DEVICE), y.to(DEVICE)
            outputs = model(x)
            preds = outputs.argmax(dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())
    
    # Compute accuracy
    acc = np.mean(np.array(all_preds) == np.array(all_labels))
    print(f"Test Accuracy: {acc:.4f}\n")

    # Convert integer labels back to string labels
    all_preds_str = label_encoder.inverse_transform(all_preds)
    all_labels_str = label_encoder.inverse_transform(all_labels)

    # Classification report
    print("Classification Report:")
    print(classification_report(all_labels_str, all_preds_str))

    # Confusion matrix
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels_str, all_preds_str))

# ==============================
# RUN
# ==============================
train_model(model, train_loader, val_loader, EPOCHS, PATIENCE)
evaluate_model(model, test_loader, le)

Using device: cpu
Found 5568 samples
Number of gloss classes after filtering: 146
CNN1D(
  (conv_block): Sequential(
    (0): Conv1d(306, 256, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.1, inplace=False)
    (4): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=(1,))
    (5): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): Dropout(p=0.05, inplace=False)
    (8): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (9): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): AdaptiveMaxPool1d(output_size=1)
  )
  (fc): Linear(in_features=64, out_features=146, bias=True)
)


Epoch 1/60 [Train]: 100%|████████████████████████████████████████████████████████████| 279/279 [00:20<00:00, 13.45it/s]


Epoch 1/60 | Train Loss: 4.7087 | Train Acc: 0.0317 | Val Acc: 0.0503
  🔹 Saved best model


Epoch 2/60 [Train]: 100%|████████████████████████████████████████████████████████████| 279/279 [00:22<00:00, 12.22it/s]


Epoch 2/60 | Train Loss: 3.8988 | Train Acc: 0.0986 | Val Acc: 0.0934
  🔹 Saved best model


...

Epoch 47/60 | Train Loss: 0.0993 | Train Acc: 0.9695 | Val Acc: 0.7899


Epoch 48/60 [Train]: 100%|███████████████████████████████████████████████████████████| 279/279 [00:25<00:00, 10.77it/s]


Epoch 48/60 | Train Loss: 0.0920 | Train Acc: 0.9697 | Val Acc: 0.7864


Epoch 49/60 [Train]: 100%|███████████████████████████████████████████████████████████| 279/279 [00:24<00:00, 11.50it/s]


Epoch 49/60 | Train Loss: 0.0908 | Train Acc: 0.9758 | Val Acc: 0.7882
Early stopping triggered!
Training finished. Best val acc: 0.7989
Test Accuracy: 0.7738

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.33      0.50         6
           1       1.00      0.33      0.50         3
           2       1.00      1.00      1.00         3
           3       0.50      1.00      0.67         3
           4       0.50      0.33      0.40         3
           5       0.83      0.83      0.83         6
           6       1.00      0.67      0.80         3
           7       1.00      1.00      1.00         3
           8       1.00      1.00      1.00         3
           9       1.00      1.00      1.00         3
          10       1.00      1.00      1.00         3
          11       0.60      1.00      0.75         3
          12       1.00      1.00      1.00         3
          13       1.00      1.00      1.00         3
      

In [10]:
import sys
!"{sys.executable}" -m pip install torchinfo --upgrade



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: C:\Users\marha\Desktop\New folder (2)\venv310\Scripts\python.exe -m pip install --upgrade pip


In [23]:
from torchinfo import summary
import os
import matplotlib.pyplot as plt

# Make sure assets folder exists
os.makedirs("assets", exist_ok=True)

# -----------------------------
# PyTorch model summary (like Keras)
# -----------------------------
model_summary = summary(
    model,
    input_size=(1, FEATURE_DIM, TARGET_FRAMES),  # batch size = 1
    col_names=["input_size", "output_size", "num_params", "trainable"],
    verbose=1,  # gives a nice tree format similar to Keras
)

# -----------------------------
# Save as UTF-8 text file
# -----------------------------
summary_file = "assets/cnn_model_summary.txt"
with open(summary_file, "w", encoding="utf-8") as f:  # <- UTF-8 fixes Windows Unicode errors
    f.write(str(model_summary))

print(f"✅ Model summary saved as text (UTF-8): {summary_file}")

# -----------------------------
# Save summary as PNG image (without displaying)
# -----------------------------
with open(summary_file, "r", encoding="utf-8") as f:
    lines = f.readlines()

fig, ax = plt.subplots(figsize=(12, len(lines)*0.3))
ax.axis("off")

for i, line in enumerate(lines):
    ax.text(0, 1 - i*0.03, line.rstrip(), fontsize=10, fontfamily='monospace')

plt.tight_layout()
png_file = "assets/cnn_model_summary.png"
plt.savefig(png_file, dpi=300, bbox_inches='tight')
plt.close(fig)  # <- do not show image in notebook

print(f"✅ Model summary saved as PNG image: {png_file}")


Layer (type:depth-idx)                   Input Shape               Output Shape              Param #                   Trainable
CNN1D                                    [1, 306, 157]             [1, 146]                  --                        True
├─Sequential: 1-1                        [1, 306, 157]             [1, 64, 1]                --                        True
│    └─Conv1d: 2-1                       [1, 306, 157]             [1, 256, 157]             235,264                   True
│    └─BatchNorm1d: 2-2                  [1, 256, 157]             [1, 256, 157]             512                       True
│    └─ReLU: 2-3                         [1, 256, 157]             [1, 256, 157]             --                        --
│    └─Dropout: 2-4                      [1, 256, 157]             [1, 256, 157]             --                        --
│    └─Conv1d: 2-5                       [1, 256, 157]             [1, 128, 157]             98,432                    True
│    └─